In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
!pip install torchsummary

In [ ]:
import os
import random
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from  sklearn.model_selection import train_test_split
%matplotlib inline

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torchvision import models, transforms
from torchsummary import summary

In [ ]:
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

In [ ]:
df_train = pd.read_csv("/kaggle/input/Kannada-MNIST/train.csv")
df_submission = pd.read_csv("/kaggle/input/Kannada-MNIST/test.csv")
df_test = pd.read_csv("/kaggle/input/Kannada-MNIST/Dig-MNIST.csv")

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
df_submission.head()

In [ ]:
rows = 5
cols = 10
fig, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(cols, rows))

for label in range(cols):
    digits = df_train.loc[df_train["label"] == label]
    digits = digits.drop("label", axis=1)
    ax[0][label].set_title(label)
    for j in range(rows):
        ax[j][label].axis("off")
        ax[j][label].imshow(digits.iloc[j, :].to_numpy().astype(np.uint8).reshape(28, 28), cmap="gray")

In [ ]:
rows = 5
cols = 10
fig, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(cols, rows))

for label in range(cols):
    digits = df_test.loc[df_test["label"] == label]
    digits = digits.drop("label", axis=1)
    ax[0][label].set_title(label)
    for j in range(rows):
        ax[j][label].axis("off")
        ax[j][label].imshow(digits.iloc[j, :].to_numpy().astype(np.uint8).reshape(28, 28), cmap="gray")

In [ ]:
print("train:{}".format(df_train.shape))
print("test:{}".format(df_test.shape))
print("submission:{}".format(df_submission.shape))

In [ ]:
print("train:{}\n\ntest:{}".format(df_train["label"].value_counts(), df_test["label"].value_counts()))

In [ ]:
train_labels = df_train["label"]
train_data = df_train.drop(columns="label")

test_labels = df_test["label"]
test_data = df_test.drop(columns="label")

submission_id = df_submission["id"]
submission_data = df_submission.drop(columns="id")

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train_data, train_labels, test_size=0.2)

In [ ]:
X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)

X_val.reset_index(drop=True, inplace=True)
y_val.reset_index(drop=True, inplace=True)

In [ ]:
resize = 28
image_size = 28

train_trans = transforms.Compose([
    transforms.ToPILImage(),
    # transforms.Resize(resize),
    transforms.RandomCrop(image_size),
    transforms.RandomAffine(degrees=5, translate=(0.1, 0.1)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])

val_trans = transforms.Compose([
    transforms.ToPILImage(),
    # transforms.Resize(resize),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])


In [ ]:
class KannadaDataSet(torch.utils.data.Dataset):
    def __init__(self, images, labels,transforms = None):
        self.X = images
        self.y = labels
        self.transforms = transforms
         
    def __len__(self):
        return (len(self.X))
    
    def __getitem__(self, i):
        data = self.X.iloc[i,:]
        data = np.array(data).astype(np.uint8).reshape(image_size,image_size,1)
        
        if self.transforms:
            data = self.transforms(data)
            
        if self.y is not None:
            return (data, self.y[i])
        else:
            return data


In [ ]:
train_ds = KannadaDataSet(X_train, y_train, train_trans)
val_ds = KannadaDataSet(X_val, y_val, val_trans)

test_ds = KannadaDataSet(test_data, test_labels, val_trans)

submission_ds = KannadaDataSet(submission_data, None, val_trans)

In [ ]:
batch_size = 256

train_loader = data.DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = data.DataLoader(val_ds, batch_size=batch_size, shuffle=False)

test_loader = data.DataLoader(test_ds, batch_size=batch_size, shuffle=False)

submission_loader = data.DataLoader(submission_ds, batch_size=batch_size, shuffle=False)

dataloaders_dict = {"train": train_loader, "val": test_loader}

In [ ]:
net = models.resnet50(pretrained=True)
net.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
# net.fc =  nn.Linear(in_features=2048, out_features=124, bias=True)
# net.relufc = nn.ReLU()
net.out =  nn.Linear(in_features=1000, out_features=10, bias=True)
net = net.to(device)
summary(net, (1, 28, 28))

In [ ]:
net

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0005)

In [ ]:
train_output = []
val_output = []

def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))

        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()
            else:
                net.eval()

            epoch_loss = 0.0
            epoch_corrects = 0

            if (epoch == 0) and (phase == 'train'):
                continue

            for inputs, labels in tqdm(dataloaders_dict[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    epoch_loss += loss.item() * inputs.size(0)
                    epoch_corrects += torch.sum(preds == labels.data)

            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)
            
            if phase == "train":
                temp_train_output = [epoch + 1, num_epochs, epoch_loss, epoch_acc]
                train_output.append(temp_train_output)
            else:
                temp_val_output = [epoch + 1, num_epochs, epoch_loss, epoch_acc]
                val_output.append(temp_val_output)
                
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))


In [ ]:
num_epochs = 50
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

In [ ]:
train_history = pd.DataFrame(train_output, columns = ["epoch","total_epochs","train_loss","train_acc"])
display(train_history)

In [ ]:
val_history = pd.DataFrame(val_output, columns = ["epoch", "total_epochs", "val_loss", "val_acc"])
display(val_history)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))
axes[0].plot(val_history['epoch'],val_history['val_loss'], label='validation_loss')
axes[0].plot(train_history['epoch'],train_history['train_loss'], label='train_loss')

axes[0].legend()

axes[1].plot(val_history['epoch'],val_history['val_acc'], label='validation_acc')
axes[1].plot(train_history['epoch'],train_history['train_acc'], label='train_acc')

axes[1].legend()

In [ ]:
net.eval()
predictions = []

for data in submission_loader:
    data = data.to(device)
    output = net(data).max(dim=1)[1]
    predictions += list(output.data.cpu().numpy())

In [ ]:
submission = pd.read_csv('../input/Kannada-MNIST/sample_submission.csv')
submission['label'] = predictions
submission.to_csv('prediction.csv', index=False)
submission.head()